In [17]:
import os

os.makedirs("docs", exist_ok=True)

print("docs/ 目录已准备好。请把所有 .md 文件放在 docs/ 下面（至少要有 index.md）。")
print("当前 docs 目录下的文件：", os.listdir("docs"))


docs/ 目录已准备好。请把所有 .md 文件放在 docs/ 下面（至少要有 index.md）。
当前 docs 目录下的文件： ['main.md', 'music.md', 'podcast.md', 'citywalk.md', 'drama.md', 'variety.md']


In [18]:
import os
import shutil
import re

SRC_DIR = "docs"       # 你的源 md 文件
OUT_DIR = "gen_docs"   # 自动生成 + 自动加链接的 md，用于 mkdocs

# ================================
# 0. 检查并准备目录
# ================================
if not os.path.isdir(SRC_DIR):
    raise FileNotFoundError("请先创建 docs/ 目录并放入 index.md 等源文件。")

os.makedirs(OUT_DIR, exist_ok=True)

# 清空 gen_docs
for f in os.listdir(OUT_DIR):
    path = os.path.join(OUT_DIR, f)
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

print("gen_docs/ 已清空并准备好。")

# ================================
# 1. 拷贝 md 源文件 → gen_docs
# ================================
md_files = [f for f in os.listdir(SRC_DIR) if f.lower().endswith(".md")]
md_files.sort()
priority_page = "A Quiet Moments.md"
if priority_page in md_files:
    md_files.remove(priority_page)
    md_files.insert(0, priority_page)
if "index.md" not in [f.lower() for f in md_files]:
    raise FileNotFoundError("docs 中必须包含 index.md 作为首页。")

for f in md_files:
    shutil.copy(os.path.join(SRC_DIR, f), os.path.join(OUT_DIR, f))

print("源 md 文件已复制到 gen_docs/。")

# ================================
# 2. 读取标题
# ================================
titles = {}

for fname in md_files:
    path = os.path.join(OUT_DIR, fname)
    title = None
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip().startswith("#"):
                title = line.strip().lstrip("#").strip()
                break
    if not title:
        title = os.path.splitext(fname)[0]
    titles[fname] = title

print("已读取页面标题。")

# ================================
# 3. 自动生成 mkdocs.yml
# ================================
yml = []
nav_files = [f for f in md_files if f.lower() != "index.md"]

# 如果你要让 A Quiet Moments 排在第一个（可选）
priority_title = "A Quiet Moments"
priority_file = None

for f in nav_files:
    if titles[f] == priority_title:
        priority_file = f
        break

if priority_file:
    nav_files.remove(priority_file)
    nav_files.insert(0, priority_file)

# 加入 nav
for fname in nav_files:
    yml.append(f"  - {titles[fname]}: {fname}")

yml.append("theme:")
yml.append("  name: material")

with open("mkdocs.yml", "w", encoding="utf-8") as f:
    f.write("\n".join(yml))

print("mkdocs.yml 已生成。")

# ================================
# 4. 自动给 gen_docs/*.md 添加链接
# ================================
AUTO_START = "<!-- AUTO-LINKS-START -->"
AUTO_END = "<!-- AUTO-LINKS-END -->"

pattern = re.compile(r"\n?\s*<!-- AUTO-LINKS-START -->.*?<!-- AUTO-LINKS-END -->", re.S)

for fname in md_files:
    path = os.path.join(OUT_DIR, fname)
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 删除旧自动区块
    content = pattern.sub("", content).rstrip() + "\n\n"

    if fname.lower() == "index.md":
        others = [(titles[f], f) for f in md_files if f.lower() != "index.md"]
        links = "\n".join(f"- [{t}]({f})" for t, f in others) or "（暂无子页面）"

        auto = (
            f"{AUTO_START}\n\n"
            "---\n\n"
            "## Page Navigation\n\n"
            f"{links}\n\n"
            f"{AUTO_END}\n"
        )
    else:
        others = [(titles[f], f) for f in md_files if f != fname]
        links = "\n".join(f"- [{t}]({f})" for t, f in others)

        auto = (
            f"{AUTO_START}\n\n"
            "---\n\n"
            "[Back to Home](index.md)\n\n"
            "## More Pages\n\n"
            f"{links}\n\n"
            f"{AUTO_END}\n"
        )

    with open(path, "w", encoding="utf-8") as f:
        f.write(content + auto)

print("gen_docs/ 中所有 md 已自动插入链接。")

print("\n✨ 已完成：复制、生成链接、生成 mkdocs.yml。接下来运行 mkdocs build。")


gen_docs/ 已清空并准备好。


FileNotFoundError: docs 中必须包含 index.md 作为首页。

In [ ]:
import sys, subprocess
result = subprocess.run(
    [sys.executable, "-m", "mkdocs", "build"],
    capture_output=True,
    text=True
)
print("stdout:\n", result.stdout)
print("stderr:\n", result.stderr)


stdout:
 
stderr:
 Error: MkDocs encountered an error parsing the configuration file: expected '<document start>', but found '<block mapping start>'
  in "g:\My Drive\Github_chan\chan\mkdocs.yml", line 6, column 1



In [ ]:
import os
import shutil

src = "site"
dst = "."

for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)

    # 删除旧文件/文件夹
    if os.path.isdir(d):
        shutil.rmtree(d)
    elif os.path.exists(d):
        os.remove(d)

    # 复制新的
    if os.path.isdir(s):
        shutil.copytree(s, d)
    else:
        shutil.copy2(s, d)

print("网站文件已复制到根目录。")


网站文件已复制到根目录。
